# Test LL SSH - Pt2

In [24]:
import pandas as pd
import os
import numpy as np
import statsmodels.discrete.discrete_model as smd
from statsmodels.tools import add_constant
import statsmodels.formula.api as smf

**Import data and format**

In [3]:
# import data
df_raw = pd.read_csv("CyclingPerceptionData-Assignment 1-2024.csv")
df_raw.head()

,ID,sex,age,OwnBikes,NumberOfBikes,HH_Adults_15+,HH_Children_Dummy,NoOf15yrsOlderBikers,BikingForAllReason,BikingForWork,...,Time from MajorBikePath_min,Time from Neasrest BikeLane_Min,Neighbourhood ID,Neighbourhood-Area_SQKM,Neighbourhood-bikeLength_KM,Neighbourhood_Population,Rank_Overall-Quality-Of-Biking,Rank_On-Street-BikeLanes-Quality,Rank_OffStreet-BikeLanes-Quality,Rank_Bike-Parking-Quality
0,40612,0,43,1,2,2,1,1,1,0,...,15,15,124,3.227141,0.763362,22387,3,3,4,5
1,40472,1,29,1,2,4,1,2,1,0,...,5,10,8,2.917358,3.234139,9437,3,1,3,3
2,40897,1,33,1,4,4,1,4,3,0,...,0,0,77,5.960673,1.037788,14830,3,4,4,5
3,40594,1,34,1,3,2,2,2,2,2,...,2,0,43,1.436087,4.510762,13777,4,3,3,4
4,41451,1,24,1,3,5,1,3,1,1,...,25,10,131,37.609169,33.950589,56281,3,4,3,4


In [101]:
# some data processing - largely guesswork
df_proc = df_raw.copy()

# set all "unknown" to 0 ("no") and convert to binary (1=yes, 0=no/unknown)
# assume bikingfor[reason] is meant to be also following the 1=yes, 2=no, 3=unknown scheme
for col in ['AccessToMotorVehicle_1Yes-2No-3Unknown', 'BikeParkingAt Work_1Yes2No3Unknown']:
    df_proc[col] = (df_proc[col] == 1).astype(int)

# assume HH_Children_Dummy is meant to be 1=yes, 2=no
df_proc['HH_Children_Dummy'] = df_proc['HH_Children_Dummy'] % 2

In [4]:
# list all provided vars
print('\n'.join(x for x in df_raw.columns if not x.startswith("Rank_")))

ID
sex
age
OwnBikes
NumberOfBikes
HH_Adults_15+
HH_Children_Dummy
NoOf15yrsOlderBikers
BikingForAllReason
BikingForWork
BikingForSchool
BikingForErrndFriends
Num of Recreational-Biker
AccessToMotorVehicle_1Yes-2No-3Unknown
BikeParkingAt Work_1Yes2No3Unknown
How longLivedInNH
Home_1Rent2Own
d3: Home Type
d5: LevelOfEducation
d6: Job Stastus
d8: Income
Time from MajorBikePath_min
Time from Neasrest BikeLane_Min
Neighbourhood ID
Neighbourhood-Area_SQKM
Neighbourhood-bikeLength_KM
Neighbourhood_Population


## Part 1. Binary Probit Model
A dataset: “CyclingPerceptionData-Assignment 1-2024,” is uploaded. 

Use column AB of sheet “Final”, which is of “Rank_Overall-Quality-Of-Biking”. 

Please consider this variable as categorical data. 

Please model the ranking over 2 as a binary Probit model. 

Please select the explanatory variables from this data sheet as relevant you think

For estimation, you can use any software. The report should include at least the following sections:
* Present the model formulations and justification of independent variables. 
* Selection of the best model. 
* Justification of using the explanatory variables 

The report should be professional, not more than 2 pages (singles-space with a 1.25-inch margin on all four sides).

**Prepare data**

In [5]:
# define Y-variable: "Rank_Overall-Quality-Of-Biking" (as categorical)
y_probit = df_raw['Rank_Overall-Quality-Of-Biking'].astype('category')
y_probit

0      3
1      3
2      3
3      4
4      3
      ..
238    5
239    5
240    2
241    3
242    3
Name: Rank_Overall-Quality-Of-Biking, Length: 243, dtype: category
Categories (6, int64): [1, 2, 3, 4, 5, 6]

In [6]:
# convert to binary: y > 2
y_probit_gt2 = (y_probit.astype('int') > 2).astype(int)
y_probit_gt2

0      1
1      1
2      1
3      1
4      1
      ..
238    1
239    1
240    0
241    1
242    1
Name: Rank_Overall-Quality-Of-Biking, Length: 243, dtype: int64

**Select relevant explanatory variables**

See above for list of all non-rank [non-response] features provided in the dataset.

In [29]:
y_probit_gt2.shape

(243,)

In [31]:
probit_Nmod = smd.Probit(y_probit_gt2, np.ones(y_probit_gt2.shape))
probit_Nres = probit_Nmod.fit()
probit_null_aic = probit_Nres.aic
probit_null_aic

Optimization terminated successfully.
         Current function value: 0.508286
         Iterations 5


249.02714800340607

In [88]:
probit_vars = ['age', 'sex', 'BikingForWork', 'Num of Recreational-Biker', 
               'AccessToMotorVehicle_1Yes-2No-3Unknown', 
               'Neighbourhood-bikeLength_KM', 'Time from MajorBikePath_min']

probit_mod = smd.Probit(y_probit_gt2, 
                        add_constant(df_raw[probit_vars], has_constant='add')
                       )
probit_res = probit_mod.fit()

print(f"{probit_res.aic:.2f} vs null {probit_null_aic:.2f} (lower is better)")
probit_res.summary()

Optimization terminated successfully.
         Current function value: 0.485867
         Iterations 6
252.13 vs null 249.03 (lower is better)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                Probit Regression Results                                 
==========================================================================================
Dep. Variable:     Rank_Overall-Quality-Of-Biking   No. Observations:                  243
Model:                                     Probit   Df Residuals:                      235
Method:                                       MLE   Df Model:                            7
Date:                            Sat, 24 Feb 2024   Pseudo R-squ.:                 0.04411
Time:                                    13:14:38   Log-Likelihood:                -118.07
converged:                                   True   LL-Null:                       -123.51
Covariance Type:                        nonrobust   LLR p-value:                    0.1432
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                      1.2650      0.466      2.713      0.007       0.351       2.179
age                                       -0.0100      0.007     -1.518      0.129      -0.023       0.003
sex                                        0.0381      0.192      0.198      0.843      -0.339       0.415
BikingForWork                              0.3104      0.151      2.049      0.040       0.014       0.607
Num of Recreational-Biker                 -0.1080      0.103     -1.050      0.294      -0.310       0.094
AccessToMotorVehicle_1Yes-2No-3Unknown    -0.1147      0.081     -1.410      0.159      -0.274       0.045
Neighbourhood-bikeLength_KM                0.0180      0.017      1.088      0.277      -0.014       0.050
Time from MajorBikePath_min                0.0130      0.012      1.057      0.290      -0.011       0.037
==========================================================================================================
"""

In [90]:
probit_vars = ['age', 'sex', 'BikingForWork', 'Num of Recreational-Biker', 
               'AccessToMotorVehicle_1Yes-2No-3Unknown', 
               'Neighbourhood-bikeLength_KM', 'Time from MajorBikePath_min']

probit_mod = smd.Probit(y_probit_gt2, 
                        add_constant(df_proc[probit_vars], has_constant='add')
                       )
probit_res = probit_mod.fit()

print(f"{probit_res.aic:.2f} vs null {probit_null_aic:.2f} (lower is better)")
probit_res.summary()

Optimization terminated successfully.
         Current function value: 0.495211
         Iterations 6
256.67 vs null 249.03 (lower is better)


/home/tiggy/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                Probit Regression Results                                 
==========================================================================================
Dep. Variable:     Rank_Overall-Quality-Of-Biking   No. Observations:                  243
Model:                                     Probit   Df Residuals:                      235
Method:                                       MLE   Df Model:                            7
Date:                            Sat, 24 Feb 2024   Pseudo R-squ.:                 0.02572
Time:                                    13:14:57   Log-Likelihood:                -120.34
converged:                                   True   LL-Null:                       -123.51
Covariance Type:                        nonrobust   LLR p-value:                    0.4990
==========================================================================================================
                                             coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
const                                      1.0649      0.396      2.691      0.007       0.289       1.841
age                                       -0.0114      0.006     -1.759      0.079      -0.024       0.001
sex                                        0.0461      0.192      0.240      0.810      -0.330       0.422
BikingForWork                              0.1702      0.189      0.902      0.367      -0.200       0.540
Num of Recreational-Biker                 -0.0578      0.101     -0.572      0.567      -0.256       0.140
AccessToMotorVehicle_1Yes-2No-3Unknown     0.1210      0.191      0.633      0.527      -0.254       0.496
Neighbourhood-bikeLength_KM                0.0173      0.017      1.034      0.301      -0.015       0.050
Time from MajorBikePath_min                0.0118      0.012      0.963      0.335      -0.012       0.036
==========================================================================================================
"""

In [96]:
probit_vars = ['BikingForWork', 'NumberOfBikes', 'd8: Income', 'HH_Children_Dummy', 'age']

probit_mod = smd.Probit(y_probit_gt2, 
                        add_constant(df_raw[probit_vars], has_constant='add')
                       )
probit_res = probit_mod.fit()

print(f"{probit_res.aic:.2f} vs null {probit_null_aic:.2f} (lower is better)")
probit_res.summary()

Optimization terminated successfully.
         Current function value: 0.484317
         Iterations 5
247.38 vs null 249.03 (lower is better)


/home/tiggy/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                Probit Regression Results                                 
==========================================================================================
Dep. Variable:     Rank_Overall-Quality-Of-Biking   No. Observations:                  243
Model:                                     Probit   Df Residuals:                      237
Method:                                       MLE   Df Model:                            5
Date:                            Sat, 24 Feb 2024   Pseudo R-squ.:                 0.04716
Time:                                    13:16:08   Log-Likelihood:                -117.69
converged:                                   True   LL-Null:                       -123.51
Covariance Type:                        nonrobust   LLR p-value:                   0.03993
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 1.6681      0.568      2.935      0.003       0.554       2.782
BikingForWork         0.2709      0.155      1.747      0.081      -0.033       0.575
NumberOfBikes        -0.0419      0.092     -0.457      0.648      -0.222       0.138
d8: Income            0.0409      0.048      0.860      0.390      -0.052       0.134
HH_Children_Dummy    -0.4370      0.219     -2.000      0.046      -0.865      -0.009
age                  -0.0079      0.007     -1.215      0.225      -0.021       0.005
=====================================================================================
"""

In [99]:
probit_vars = ['BikingForWork', 'NumberOfBikes', 'HH_Children_Dummy', 'age', 'Neighbourhood-bikeLength_KM']

probit_mod = smd.Probit(y_probit_gt2, 
                        add_constant(df_proc[probit_vars], has_constant='add')
                       )
probit_res = probit_mod.fit()

print(f"{probit_res.aic:.2f} vs null {probit_null_aic:.2f} (lower is better)")
probit_res.summary()

Optimization terminated successfully.
         Current function value: 0.488827
         Iterations 6
249.57 vs null 249.03 (lower is better)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                Probit Regression Results                                 
==========================================================================================
Dep. Variable:     Rank_Overall-Quality-Of-Biking   No. Observations:                  243
Model:                                     Probit   Df Residuals:                      237
Method:                                       MLE   Df Model:                            5
Date:                            Sat, 24 Feb 2024   Pseudo R-squ.:                 0.03828
Time:                                    13:16:44   Log-Likelihood:                -118.79
converged:                                   True   LL-Null:                       -123.51
Covariance Type:                        nonrobust   LLR p-value:                   0.09216
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           0.9060      0.372      2.439      0.015       0.178       1.634
BikingForWork                   0.1483      0.189      0.783      0.433      -0.223       0.519
NumberOfBikes                   0.0136      0.084      0.162      0.871      -0.152       0.179
HH_Children_Dummy               0.4079      0.215      1.900      0.057      -0.013       0.829
age                            -0.0096      0.006     -1.523      0.128      -0.022       0.003
Neighbourhood-bikeLength_KM     0.0173      0.017      1.023      0.307      -0.016       0.051
===============================================================================================
"""

In [100]:
probit_vars = ['BikingForAllReason', 'NumberOfBikes', 'HH_Children_Dummy', 'age', 'Neighbourhood-bikeLength_KM']

probit_mod = smd.Probit(y_probit_gt2, 
                        add_constant(df_proc[probit_vars], has_constant='add')
                       )
probit_res = probit_mod.fit()

print(f"{probit_res.aic:.2f} vs null {probit_null_aic:.2f} (lower is better)")
probit_res.summary()

Optimization terminated successfully.
         Current function value: 0.489560
         Iterations 6
249.93 vs null 249.03 (lower is better)


/home/tiggy/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                Probit Regression Results                                 
==========================================================================================
Dep. Variable:     Rank_Overall-Quality-Of-Biking   No. Observations:                  243
Model:                                     Probit   Df Residuals:                      237
Method:                                       MLE   Df Model:                            5
Date:                            Sat, 24 Feb 2024   Pseudo R-squ.:                 0.03684
Time:                                    13:16:59   Log-Likelihood:                -118.96
converged:                                   True   LL-Null:                       -123.51
Covariance Type:                        nonrobust   LLR p-value:                    0.1051
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                           1.0187      0.382      2.668      0.008       0.270       1.767
BikingForAllReason             -0.1100      0.216     -0.509      0.611      -0.533       0.313
NumberOfBikes                   0.0063      0.088      0.072      0.943      -0.166       0.179
HH_Children_Dummy               0.4208      0.216      1.945      0.052      -0.003       0.845
age                            -0.0090      0.006     -1.392      0.164      -0.022       0.004
Neighbourhood-bikeLength_KM     0.0190      0.017      1.105      0.269      -0.015       0.053
===============================================================================================
"""

In [38]:
print('\n'.join(df_raw.columns))

ID
sex
age
OwnBikes
NumberOfBikes
HH_Adults_15+
HH_Children_Dummy
NoOf15yrsOlderBikers
BikingForAllReason
BikingForWork
BikingForSchool
BikingForErrndFriends
Num of Recreational-Biker
AccessToMotorVehicle_1Yes-2No-3Unknown
BikeParkingAt Work_1Yes2No3Unknown
How longLivedInNH
Home_1Rent2Own
d3: Home Type
d5: LevelOfEducation
d6: Job Stastus
d8: Income
Time from MajorBikePath_min
Time from Neasrest BikeLane_Min
Neighbourhood ID
Neighbourhood-Area_SQKM
Neighbourhood-bikeLength_KM
Neighbourhood_Population
Rank_Overall-Quality-Of-Biking
Rank_On-Street-BikeLanes-Quality
Rank_OffStreet-BikeLanes-Quality
Rank_Bike-Parking-Quality


## Part 2. Binary Logit Model:
A dataset: “CyclingPerceptionData-Assignment 1-2024,” is uploaded. 

Use column AB of sheet “Final”, which is of “Rank_On-Street-BikeLanes-Quality”. 

Please consider this variable as categorical data. 

Please model the ranking over 1 as a binary Logit model. 

Please select the explanatory variables from this data sheet as relevant you think

For estimation, you can use any software. The report should include at least the following sections:
* Present the model formulations and justification of independent variables. 
* Selection of the best model. 
* Justification of using the explanatory variable

The report should be professional, not more than 2 pages (singles-space with a 1.25-inch margin on all four sides).

**Prepare data**

In [6]:
# define Y-variable: "Rank_On-Street-BikeLanes-Quality" (as categorical)
y_logit = df_raw['Rank_On-Street-BikeLanes-Quality'].astype('category')
y_logit

0      3
1      1
2      4
3      3
4      4
      ..
238    5
239    5
240    2
241    4
242    4
Name: Rank_On-Street-BikeLanes-Quality, Length: 243, dtype: category
Categories (6, int64): [1, 2, 3, 4, 5, 6]

In [7]:
# convert to binary: Y > 1
y_logit_gt1 = (y_logit.astype(int) > 1).astype(int)
y_logit_gt1

0      1
1      0
2      1
3      1
4      1
      ..
238    1
239    1
240    1
241    1
242    1
Name: Rank_On-Street-BikeLanes-Quality, Length: 243, dtype: int64

**Select relevant explanatory variables**

See above for list of all non-rank [non-response] features provided in the dataset.

In [ ]:
# binary_logistic = smf.logit(formula, data = df).fit()
# binary_logistic.summary()